# Using TMPL in a Jupyter notebook

In [1]:
# Setup importing tmp if it has not been installed using pip
import pathlib
import os
import sys

BASEPATH = pathlib.Path(os.getcwd()).parent.parent
print(BASEPATH)
sys.path.append(str(BASEPATH))
sys.path.append(str(BASEPATH/'unit_test'))


import tmpl

/home/redlegjed/Documents/Python/Projects/test_measure_process_lib


# Example test sequence

In [2]:
# Import example test sequence
import example_resistor_test

In [3]:

# Define resistor model
res1 = example_resistor_test.ResistorModel(100,tolerance_pc=1.0)

# Setup resources
resources = {
    'set_temperature':example_resistor_test.set_temperature,
    'set_humidity': example_resistor_test.set_humidity,
    'voltage_supply':example_resistor_test.VoltageSupply(),
    'resistor':res1,
    }

config_custom = {'param1':1,'param2':2}

# Create and run the test sequence
test = example_resistor_test.ExampleTestSequence(resources,config=config_custom)

## HTML representation

TMPL TestManager objects have an HTML representation for Jupyter notebooks

In [4]:
test

TestManager[ExampleResistorTest]
* Conditions:
	Iteration
	temperature_degC
	humidity_pc
* Measurements:
	Timestamp
	TurnOn
	Stabilise
	VoltageSweep
	TurnOff